In [1]:
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.palettes import diverging_palette, all_palettes
from bokeh.transform import linear_cmap

import matplotlib.colors as mc

output_notebook(hide_banner=True)

from tqdm.auto import tqdm

import numpy as np
import pandas as pd

from pypop.prv import PRV

# Loading raw Extrae Trace Data

Raw trace data can be loaded using the PRV class from `pypop.prv`:

In [2]:
testfile_path="./omp_detail.prv.gz"
prv = PRV(testfile_path)

This results in a PRV object containing the events, communications and running state data in three dataframes:

In [3]:
#display(prv.state)

In [4]:
#display(prv.event)

Note that in the case of a pure OpenMP code there are no MPI communications and so the `comm` attribute will be `None`

In [5]:
#display(prv.comm)

In [6]:
omp_region_stats = prv.profile_openmp_regions()

A Jupyter Widget

In [10]:
bad_thres = 0.5
good_thres = 0.8
cmap_points = [(0.0, (
    0.690, 0.074, 0.074)),
               (bad_thres, (0.690, 0.074, 0.074)),
               (good_thres-1e-20, (0.992, 0.910, 0.910)),
               (good_thres, (0.910, 0.992, 0.910)),
               (1.0, (0.074, 0.690, 0.074))]

metric_cmap = mc.LinearSegmentedColormap.from_list('POP_Metrics', colors=cmap_points, N=256, gamma=1)
metric_palette = diverging_palette(all_palettes['Reds'][256],all_palettes['Greens'][256], 256, 0.8)

tooltip_template = [('Function(s)', "@{Region Function Fingerprint}"),
                    ('Location(s)', "@{Region Location Fingerprint}"),
                    ('Load Bal.', "@{Load Balance}"),
                    ('Length', "@{Region Length} ns"),
                    ('Avg. Comp.', "@{Average Computation Time}"),
                    ('Max. Comp.', "@{Maximum Computation Time}"),
                    ('Sum Comp.', "@{Region Total Computation}"),]

fig = figure(plot_width=900, plot_height=600, tools="xwheel_zoom,zoom_in,zoom_out,pan,reset,save", tooltips=tooltip_template)
for rank, rankdata in omp_region_stats.groupby(level='rank'):
    fig.hbar(y=rank,
             left = 'Region Start',
             right='Region End',
             height = 0.9,
             color=linear_cmap('Load Balance',metric_palette,0,1),
             source=rankdata)
    
show(fig)

In [8]:
prv.openmp_region_summary(by_location=True)

,Instances,Relative Load Balance Efficiency,Relative Parallel Efficiency,Load Balance Efficiency,Parallel Efficiency,Accumulated Region Time,Accumulated Computation Time,Average Computation Time,Maximum Computation Time,Region Functions
Region Location Fingerprint,,,,,,,,,,
"92 (pixel-accessor.h, libMagickCore-7.Q16.so.6.0.0)",16,0.388298,0.386530,0.825248,0.824743,41815551634,129303741962,1.010185e+09,6934777630,"92 (pixel-accessor.h, libMagickCore-7.Q16.so.6..."
"186 (pixel-accessor.h, libMagickCore-7.Q16.so.6.0.0)",16,0.429397,0.422728,0.941165,0.940477,15092415397,51039946618,3.987496e+08,2234636901,"186 (pixel-accessor.h, libMagickCore-7.Q16.so...."
